In [1]:
import pandas as pd
import numpy as np
import time
import math
import sklearn.model_selection
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_csv('IMBD_train.csv')

In [3]:
df.iloc[:,1] = df.iloc[:,1].str.replace('.csv', '')
df = df.drop(df.columns[0], axis=1)

In [4]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,A11887,G21_1230,20200219,PTC13,Deg.F,66.7,66.7,66.7,66.7,69.9,...,95.3,98.3,101.9,104.6,105.9,109.6,113.5,117.1,121.7,125.5
1,A11887,G21_1230,20200219,PTC14,Deg.F,66.7,66.7,66.7,66.7,68.2,...,85.1,87.8,90.7,92.9,93.9,97.0,100.2,103.8,107.0,110.8
2,A11887,G21_1230,20200219,PTC15,Deg.F,66.3,66.3,66.3,66.3,68.1,...,89.6,91.9,94.9,98.0,98.6,102.1,105.9,109.7,113.4,117.3
3,A85484,G21_1230,20200219,PTC22,Deg.F,67.7,67.7,67.7,67.7,68.9,...,82.8,85.1,87.2,89.2,89.8,92.8,95.7,98.6,101.7,104.7
4,A85484,G21_1230,20200219,PTC23,Deg.F,68.0,68.0,68.0,68.0,69.1,...,83.0,85.1,87.5,89.3,90.0,92.8,95.7,98.5,101.4,104.3
5,A85484,G21_1230,20200219,PTC24,Deg.F,68.4,68.4,68.4,68.4,69.6,...,83.0,84.9,86.9,88.9,89.4,92.0,94.6,97.3,100.1,103.0
6,A47143,G21_1230,20200219,PTC4,Deg.F,68.1,68.1,68.1,68.1,69.8,...,89.2,92.3,95.4,98.1,98.9,102.3,105.8,109.4,113.3,117.0
7,A47143,G21_1230,20200219,PTC5,Deg.F,68.4,68.4,68.4,68.4,69.9,...,88.9,91.7,94.9,97.6,98.1,101.6,105.4,108.8,112.8,116.4
8,A47143,G21_1230,20200219,PTC6,Deg.F,68.7,68.7,68.7,68.7,69.9,...,86.7,89.3,91.8,94.0,95.0,98.0,101.2,104.1,107.6,111.0
9,A11887,G21_1230,20190717,PTC10,Deg.F,87.1,87.1,87.1,87.1,88.5,...,105.0,107.9,110.5,112.9,113.5,116.3,119.5,123.2,126.5,129.8


In [5]:
workpiece = df['0'].unique()
group = df['1'].unique()
print(len(workpiece),workpiece, '\n', len(group), group)

32 ['A11887' 'A85484' 'A47143' 'A121151' 'A555894' 'A927695' 'A859833'
 'A726846' 'A37248' 'A9017' 'A35776' 'A87907589' 'A84462' 'A775826'
 'A794147' 'A330530' 'A314222' 'A393450' 'A714858' 'A834782' 'B555894'
 'B121151' 'A79630' 'B927695' 'A369919' 'A778989' 'A538130' 'A620395'
 'A913844' 'A65195' 'A511193' 'A46872'] 
 7 ['G21_1230' 'G01_715' 'G02_715' 'G15_1230' 'G13_715' 'G11-5_715'
 'G49_1030']


In [6]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2)
print(len(train_df),len(val_df))

1568 393


In [7]:
train_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
1811,A65195,G49_1030,20190104,PTC10,Deg.F,68.8,68.8,68.9,69.9,70.8,...,77.2,77.7,78.2,78.7,78.5,79.1,79.4,79.8,80.2,80.6
1338,B555894,G13_715,20200204,PTC14,Deg.F,66.3,66.3,66.3,66.2,66.8,...,87.3,91.2,95.1,99.0,103.6,107.7,112.9,117.2,121.9,126.8


In [83]:
class Dataset(Dataset):
    def __init__(self, mode, df):
        assert mode in ["train", "test"]  
        self.mode = mode
        self.index = df.iloc[:,0]
        self.data = df
        self.len = len(self.data)

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        x = self.data.iloc[idx,6:]
        if self.mode == "train":
            workpiece = list(df['0'].unique())
            group = list(df['1'].unique())
            w_name = self.data.iloc[idx][0]
            g_name = self.data.iloc[idx][1]
            
            w_index = workpiece.index(w_name)
            g_index = group.index(g_name)
            input_x = torch.tensor([x])
            
            return input_x.float(), torch.tensor([w_index]).float(), torch.tensor([g_index]).float()
        else:
            return torch.tensor([x]).float()
# input_dim = 10    
# hidden_dim = 256
# layer_dim = 3
# output_dim = 9
# seq_dim = 128

In [79]:
class CNN_Model(nn.Module):
    #列出需要哪些層
    def __init__(self, output_dim_g, output_dim_w, input_size, hidden_size, batch_size):
        super(CNN_Model, self).__init__()
        # Convolution 1 , input_shape=(
        self.cnn1 = nn.Conv1d(input_size, 256, kernel_size=5, stride=1) 
        self.relu1 = nn.ReLU(inplace=True) 
        # Max pool 1
        self.maxpool1 = nn.AvgPool2d(kernel_size=2)
        # Convolution 2
        self.cnn2 = nn.Conv1d(128, 64, kernel_size=7, stride=1) 
        self.relu2 = nn.ReLU(inplace=True) 
        # Max pool 2
        self.maxpool2 = nn.AvgPool2d(kernel_size=2)
        # Fully connected 1 ,#input_shape=(
        self.fc_g = nn.Linear(32, output_dim_g)
        self.fc_w = nn.Linear(32, output_dim_w)    
    
    def forward(self, x):
        out = self.cnn1(x) 
        out = self.relu1(out)
        out = self.maxpool1(out)
        #print(out.shape)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        #print(out.shape)
        out = out.view(out.size(0), -1) 
        #print(out.shape)
        out_g = self.fc_g(out) 
        out_w = self.fc_w(out) 
        return out_w, out_g

In [13]:
class classifier(nn.Module):
    #layers used in model
    def __init__(self, output_dim_g, output_dim_w, input_size, hidden_dim, n_layers, batch_size, drop_prob, bidirectional=True):
        super().__init__()
        self.lstm = nn.LSTM(input_size,
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=drop_prob,
                           batch_first=True)
        #output layer
        if bidirectional:
            self.fc = nn.Linear(hidden_dim * 2, 128)
        else:
            self.fc = nn.Linear(hidden_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc_g = nn.Linear(64, output_dim_g)
        self.fc_w = nn.Linear(64, output_dim_w)
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
                            
    def init_hidden(self):
        #initialise hidden state 
        return (torch.zeros(self.n_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.n_layers, self.batch_size, self.hidden_dim))
    
    def forward(self, x, h= None):
        
        out, h = self.lstm(x, h)
        out = F.leaky_relu(self.fc(out))
        out = F.leaky_relu(self.fc2(out))
        out_w = self.fc_w(out)
        out_g = self.fc_g(out)
#         tags = F.log_softmax(out, dim=1)
        return out_w.squeeze(1), out_g.squeeze(1)

In [96]:
def calc_score(accuracy):
    if accuracy >= .75:
        return 100
    elif accuracy >= .65:
        return 90
    elif accuracy >= .55:
        return 80
    elif accuracy >= .45:
        return 70
    elif accuracy >= .35:
        return 60
    else:
        return 0

In [98]:
def asMinutes(s): #s = time.time()-start_time
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def evaluation(valloader,net):
    net = net.eval()
    total, tp_w, tp_g, d = 0, 0, 0, 0
    for x, true_w, true_g in valloader:
        pred_w, pred_g = net(x)
        if torch.argmax(pred_g,dim=1) == true_g:
            tp_g += 1
        if torch.argmax(pred_w,dim=1) == true_w:
            tp_w += 1
        if torch.argmax(pred_w,dim=1) == true_w and torch.argmax(pred_g,dim=1) == true_g:
            d += 1
        total += 1
    return (tp_w/total, tp_g/total, d/total)
    
def trainIter(model, trainloader, valloader, epochs, LR):
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    loss_func = torch.nn.CrossEntropyLoss()
    start_time = time.time()
    best_score = 0
    for epoch in range(epochs):
        steps, train_loss, total_wloss, total_gloss = 0, 0, 0, 0
        for x, y_w, y_g in trainloader:
            pred_w, pred_g = model(x)
            #pred_w, pred_g = model(x.permute(2,0,1))
            #print(pred_g.shape,torch.argmax(pred_g,dim=1))
            loss_w = loss_func(pred_w, y_w.squeeze(1).long())
            loss_g = loss_func(pred_g, y_g.squeeze(1).long())
            final_loss = loss_g*0.8 + loss_w*0.2
            optimizer.zero_grad()
            final_loss.backward()
            optimizer.step()
            train_loss += final_loss.item()
            total_wloss += loss_w.item()
            total_gloss += loss_g.item()
            steps += 1
            print(f'Epoch : {epoch+1}/{epochs}, time: {asMinutes(time.time()-start_time)}, Training Loss : {train_loss/steps},Group Loss : {total_gloss/steps},WorkPiece Loss : {total_wloss/steps}', end = '\r')
        tp_w, tp_g, d = evaluation(valloader, model)
        score = calc_score(d)*0.3 + calc_score(tp_g)*0.7
        if score >= best_score:
            torch.save(model.state_dict(), f'IMBD_final_1012') 
            best_score = score
    
        print(f'Epoch : {epoch+1}/{epochs}, Accuracy : {score}/{tp_g}/{tp_w}/{d}',  end = '\r') 
        if epoch % 50 == 0:
            print('\n===============================================================================================')

In [109]:
train = Dataset('train', train_df)
trainloader = torch.utils.data.DataLoader(train, batch_size= 262, drop_last = True,shuffle=True)
for x,y_w, y_g in trainloader:
#     print(x.shape, y_w.shape)
#     print(x)
    break

In [ ]:
class W_Classifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(input_size, hidden_size)   # hidden layer
        self.predict = torch.nn.Linear(hidden_size, output_size)   # output layer
        self.dropout = torch.nn.Dropout(p=0.5)
    def forward(self, x):
        x = F.leaky_relu(self.hidden(x))  
        x = self.dropout(x)
        #x = F.leaky_relu(self.hidden2(x))
        #x = self.dropout(x)
        x = self.predict(x) 
        return x
    
#街 max -> group 
group = df['1'].unique()
g_name = group[g_index]
g_index = group.index(g_name)

if group_name == 'G21_1230':
    class_no = 3
elif group_name == 'G01_715':
    class_no = 3
elif group_name == 'G02_715':
    class_no = 7
elif group_name == 'G15_1230':
    class_no = 7
elif group_name == 'G13_715':
    class_no = 5
elif group_name == 'G11-5_715':
    class_no = 4
else: # 'G49_1030'
    class_no = 3


In [99]:
input_size = 23 
HIDDEN = 256
N = 2
BS = 256
output_dim_g = 7
output_dim_w = 32 
dropout = 0.5
model = classifier(output_dim_g=output_dim_g, output_dim_w=output_dim_w, input_size=input_size, hidden_dim=HIDDEN, n_layers=N, batch_size=BS, drop_prob=dropout)
#model = CNN_Model(output_dim_g=output_dim_g, output_dim_w=output_dim_w, input_size=input_size, hidden_size=HIDDEN, batch_size=BS)
train = Dataset('train', train_df)
trainloader = torch.utils.data.DataLoader(train, batch_size= BS, drop_last = True,shuffle=True)
val = Dataset('train', val_df)
valloader = torch.utils.data.DataLoader(val, batch_size=1, shuffle=False)
trainIter(model, trainloader, valloader, epochs=500, LR=5e-5)

Epoch : 1/500, Accuracy : 0.0/0.061068702290076333/0.015267175572519083/0.01.9639222621917725,WorkPiece Loss : 3.47404789924621644
Epoch : 51/500, Accuracy : 56.0/0.5699745547073791/0.26208651399491095/0.1195928753180661665033,WorkPiece Loss : 2.63213153680165623
Epoch : 101/500, Accuracy : 56.0/0.6183206106870229/0.3256997455470738/0.239185750636132324079742,WorkPiece Loss : 2.1743333737055465
Epoch : 151/500, Accuracy : 62.99999999999999/0.6946564885496184/0.3384223918575064/0.24936386768447838iece Loss : 1.9712968269983928
Epoch : 201/500, Accuracy : 62.99999999999999/0.6921119592875318/0.361323155216285/0.2544529262086514kPiece Loss : 1.8528268535931904
Epoch : 251/500, Accuracy : 62.99999999999999/0.7073791348600509/0.3791348600508906/0.2824427480916031Piece Loss : 1.772594948609679376
Epoch : 301/500, Accuracy : 62.99999999999999/0.7251908396946565/0.3893129770992366/0.30025445292620867Piece Loss : 1.70341394344965624
Epoch : 351/500, Accuracy : 62.99999999999999/0.74554707379134